# Module 1

Given functions

In [1]:
# implementation of the naive exact matching algorithm
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences
    

# function that takes a DNA string and returns its reverse complement
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t


# function that parses a DNA reference genome from a file in the FASTA format.
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome


# function that parses the read and quality strings from a FASTQ file containing sequencing reads.
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

First, implement a version of the naive exact matching algorithm that is strand-aware. That is, instead of looking only for occurrences of P in T, additionally look for occurrences of thereverse complement of P in T. If P is ACT, your function should find occurrences of both ACTand its reverse complement AGT in T.

If P and its reverse complement are identical (e.g. AACGTT), then a given match offset should be reported only once. So if your new function is called naive_with_rc, then the old naivefunction and your new naive_with_rc function should return the same results when P equals its reverse complement.

In [2]:
def naive_with_rc(pattern, text):
    """
    Strand-aware exact matching algorithm.
    Searches for occurrences of pattern and its reverse complement (if it's different) in text.
    Returns sorted list.
    """
    
    occurrences = set() # Using a set to counter duplicates
    
    for i in range(len(text) - len(pattern) + 1):
        match = True
        for j in range(len(pattern)):
            if text[i+j] != pattern[j]:
                match = False
                break
        if match:
            occurrences.add(i)

    revcom_pattern = reverseComplement(pattern)

    if revcom_pattern != pattern:
        for i in range(len(text) - len(revcom_pattern) + 1):
            match = True
            for j in range(len(revcom_pattern)):
                if text[i+j] != revcom_pattern[j]:
                    match = False
                    break
            if match:
                occurrences.add(i)

    return sorted(occurrences)

In [3]:
def test_naive_with_rc():
    # Example 1
    p = 'CCC'
    ten_as = 'AAAAAAAAAA'
    t = ten_as + 'CCC' + ten_as + 'GGG' + ten_as
    occurrences = naive_with_rc(p, t)
    assert occurrences == [10, 23], f"Example 1 failed: {occurrences}"
    print("Example 1 passed.")

    # Example 2
    p = 'CGCG'
    t = ten_as + 'CGCG' + ten_as + 'CGCG' + ten_as
    occurrences = naive_with_rc(p, t)
    assert occurrences == [10, 24], f"Example 2 failed: {occurrences}"
    print("Example 2 passed.")

    phix_genome = readGenome(r'C:\Users\i.mamalis\Downloads\phix.fa')
    occurrences = naive_with_rc('ATTA', phix_genome)
    leftmost = min(occurrences)
    assert leftmost == 62, f"Example 3 failed: leftmost occurrence is {leftmost}"
    print("Example 3 passed.")
    print(f"# occurrences in Example 3: {len(occurrences)}")

if __name__ == "__main__":
    test_naive_with_rc()

Example 1 passed.
Example 2 passed.
Example 3 passed.
# occurrences in Example 3: 60


Question 1:
How many times does AGGT or its reverse complement (ACCT) occur in the lambda virus genome? E.g. if AGGTAGGT occurs 10 times and ACCTACCT occurs 12 times, you should report 22.

Question 2:
How many times does TTAATTAA or its reverse complement occur in the lambda virus genome?  
Hint: TTAATTAA and its reverse complement are equal, so remember not to double count.

Question 3:
What is the offset of the leftmost occurrence of ACTAAGT or its reverse complement in the Lambda virus genome? E.g. if the leftmost occurrence of ACTAAGT is at offset 40 (0-based) and the leftmost occurrence of its reverse complement ACTTAGT is at offset 29, then report 29.

Question 4:
What is the offset of the leftmost occurrence of AGTCGA or its reverse complement in the Lambda virus genome?

In [4]:
file_path = r'C:\Users\i.mamalis\Downloads\lambda_virus.fa'
    
genome = readGenome(file_path)

# Question 1
pattern_q1 = 'AGGT'
occurrences_q1 = naive_with_rc(pattern_q1, genome)
print(f"Question 1: Number of occurrences of {pattern_q1} or its reverse complement: {len(occurrences_q1)}")

# Question 2
pattern_q2 = 'TTAA'
occurrences_q2 = naive_with_rc(pattern_q2, genome)
print(f"Question 2: Number of occurrences of {pattern_q2} or its reverse complement: {len(occurrences_q2)}")

# Question 3
pattern_q3 = 'ACTAAGT'
occurrences_q3 = naive_with_rc(pattern_q3, genome)
leftmost_q3 = occurrences_q3[0] if occurrences_q3 else None
print(f"Question 3: Leftmost occurrence of {pattern_q3} or its reverse complement: {leftmost_q3}")

# Question 4
pattern_q4 = 'AGTCGA'
occurrences_q4 = naive_with_rc(pattern_q4, genome)
leftmost_q4 = occurrences_q4[0] if occurrences_q4 else None
print(f"Question 4: Leftmost occurrence of {pattern_q4} or its reverse complement: {leftmost_q4}")

Question 1: Number of occurrences of AGGT or its reverse complement: 306
Question 2: Number of occurrences of TTAA or its reverse complement: 195
Question 3: Leftmost occurrence of ACTAAGT or its reverse complement: 26028
Question 4: Leftmost occurrence of AGTCGA or its reverse complement: 450


For Questions 5 and 6, make a new version of the naivenaive function called naive_2mm that allows up to 2 mismatches per occurrence. Unlike for the previous questions, do not consider the reverse complement here.  We're looking for approximate matches for P itself, not its reverse complement.

For example, ACTTTA occurs twice in ACTTACTTGATAAAGT, once at offset 0 with 2 mismatches, and once at offset 4 with 1 mismatch. So naive_2mm('ACTTTA', 'ACTTACTTGATAAAGT') should return the list [0, 4].

In [5]:
def naive_2mm(pattern, text):
    """
    Finds approximate matches of a pattern within a text, allowing up to 2 mismatches.
    
    Args:
        pattern (str): The pattern to search for.
        text (str): The text in which to search for the pattern.
        
    Returns:
        list: Starting indices of each match within the specified mismatch limit.
    """
    
    match_positions = []
    pattern_length = len(pattern)
    text_length = len(text)
    
    for i in range(text_length - pattern_length + 1):
        mismatches = 0
  
        for j in range(pattern_length):
            if text[i + j] != pattern[j]:
                mismatches += 1
              
                if mismatches > 2:
                    break

        if mismatches <= 2:
            match_positions.append(i)
    
    return match_positions

Question 5:
How many times does TTCAAGCCTTCAAGCC occur in the Lambda virus genome when allowing up to 2 mismatches? 

Question 6: What is the offset of the leftmost occurrence of AGGAGGTT in the Lambda virus genome when allowing up to 2 mismatches?


In [6]:
# Question 5
pattern_q5 = 'TTCAAGCC'
occurrences_q5 = naive_2mm(pattern_q5, genome)
print(f"Question 5: Number of occurrences of {pattern_q5} with up to 2 mismatches: {len(occurrences_q5)}")

# Question 6
pattern_q6 = 'AGGAGGTT'
occurrences_q6 = naive_2mm(pattern_q6, genome)
leftmost_q6 = occurrences_q6[0] if occurrences_q6 else None
print(f"Question 6: Leftmost occurrence of {pattern_q6} with up to 2 mismatches: {leftmost_q6}")

Question 5: Number of occurrences of TTCAAGCC with up to 2 mismatches: 191
Question 6: Leftmost occurrence of AGGAGGTT with up to 2 mismatches: 49



Finally, download and parse the provided FASTQ file containing real DNA sequencing reads derived from a human:

 https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR037900_1.first1000.fastq

Note that the file has many reads in it and you should examine all of them together when answering this question.  The reads are taken from this study:

Ajay, S. S., Parker, S. C., Abaan, H. O., Fajardo, K. V. F., & Margulies, E. H. (2011). Accurate

and comprehensive sequencing of personal genomes. Genome research, 21(9), 1498-1505. 

This dataset has something wrong with it; one of the sequencing cycles is poor quality.

Question 7:
Report which sequencing cycle has the problem.  Remember that a sequencing cycle corresponds to a particular offset in all the reads. For example, if the leftmost read position seems to have a problem consistently across reads, report 0. If the fourth position from the left has the problem, report 3. Do whatever analysis you think is needed to identify the bad cycle. It might help to review the "Analyzing reads by position" video.

In [7]:
def find_bad_cycle(qualities):
    """
    Find the sequencing cycle with the lowest average quality score, using the quality line of the genome file.
    """
    
    num_cycles = len(qualities[0])
    total_quality = [0] * num_cycles
    num_reads = len(qualities)

    for qual in qualities:
        for i in range(num_cycles):
            total_quality[i] += ord(qual[i]) - 33

    avg_quality_per_cycle = [total_quality[i] / num_reads for i in range(num_cycles)]

    bad_cycle = avg_quality_per_cycle.index(min(avg_quality_per_cycle))
    
    return bad_cycle

In [8]:
# Question 7
sequences, qualities = readFastq(r'C:\Users\i.mamalis\Downloads\ERR037900_1.first1000.fastq')
bad_cycle = find_bad_cycle(qualities)
print(f"Question 7: The bad sequencing cycle is at position: {bad_cycle}")

Question 7: The bad sequencing cycle is at position: 66


# Module 2

In a practical, we saw Python code implementing the Boyer-Moore algorithm. Some of the code is for preprocessing the pattern P into the tables needed to execute the bad character and good suffix rules — we did not discuss that code. But we did discuss the code that performs the algorithm given those tables:

In [9]:
# Given function
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    
    i = 0
    occurrences = []
    
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
                
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
            
        i += shift
        
    return occurrences

def readFasta(filename):
    with open(filename, 'r') as file:
        sequence = ''
        for line in file:
            if not line.startswith('>'):
                sequence += line.strip()
    return sequence

Measuring Boyer-Moore's benefit. First, download the Python module for Boyer-Moore preprocessing:

This module provides the BoyerMoore class, which encapsulates the preprocessing info used by the boyer_moore function above. Second, download the provided excerpt of human chromosome 1:

Third, implement versions of the naive exact matching and Boyer-Moore algorithms that additionally count and return (a) the number of character comparisons performed and (b) the number of alignments tried. Roughly speaking, these measure how much work the two different algorithms are doing.

In [10]:
from bm_preproc import BoyerMoore
from kmer_index import Index

In [11]:
def naive_with_counts(p, t):
    """Naive exact matching algorithm with counts of character comparisons and alignments."""
    
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    
    for i in range(len(t) - len(p) + 1):
        
        num_alignments += 1
        match = True
        
        for j in range(len(p)):
            num_character_comparisons += 1
            if t[i + j] != p[j]:
                match = False
                break
                
        if match:
            occurrences.append(i)
            
    return occurrences, num_alignments, num_character_comparisons


def boyer_moore_with_counts(p, p_bm, t):
    """Boyer-Moore matching algorithm with counts of character comparisons and alignments."""
    
    i = 0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    
    while i < len(t) - len(p) + 1:
        num_alignments += 1
        shift = 1
        mismatched = False
        
        for j in range(len(p)-1, -1, -1):
            num_character_comparisons += 1
            
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
                
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs, 1)
            
        i += shift
        
    return occurrences, num_alignments, num_character_comparisons

In [12]:
def test_boyer_moore_and_naive_with_counts():
    lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '

    # Example 1 | Boyer-Moore
    p = 'word'
    t = 'there would have been a time for such a word'
    p_bm = BoyerMoore(p, lowercase_alphabet)
    occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
    assert occurrences == [40], f"Boyer-Moore Example 1 failed: {occurrences}"
    assert num_alignments == 12, f"Boyer-Moore Example 1 failed: {num_alignments}"
    assert num_character_comparisons == 15, f"Boyer-Moore Example 1 failed: {num_character_comparisons}"
    print("Boyer-Moore Example 1 passed.")
    print(f"Occurrences: {occurrences}")
    print(f"Number of alignments tried: {num_alignments}")
    print(f"Number of character comparisons: {num_character_comparisons}\n")

    # Example 2 | Boyer-Moore
    p = 'needle'
    t = 'needle need noodle needle'
    p_bm = BoyerMoore(p, lowercase_alphabet)
    occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
    assert occurrences == [0, 19], f"Boyer-Moore Example 2 failed: {occurrences}"
    assert num_alignments == 5, f"Boyer-Moore Example 2 failed: {num_alignments}"
    assert num_character_comparisons == 18, f"Boyer-Moore Example 2 failed: {num_character_comparisons}"
    print("Boyer-Moore Example 2 passed.")
    print(f"Occurrences: {occurrences}")
    print(f"Number of alignments tried: {num_alignments}")
    print(f"Number of character comparisons: {num_character_comparisons}\n")

    # Example 1 | Naive Matching
    p = 'word'
    t = 'there would have been a time for such a word'
    occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
    assert occurrences == [40], f"Naive Matching Example 1 failed: {occurrences}"
    assert num_alignments == 41, f"Naive Matching Example 1 failed: {num_alignments}"
    assert num_character_comparisons == 46, f"Naive Matching Example 1 failed: {num_character_comparisons}"
    print("Naive Matching Example 1 passed.")
    print(f"Occurrences: {occurrences}")
    print(f"Number of alignments tried: {num_alignments}")
    print(f"Number of character comparisons: {num_character_comparisons}\n")

    # Example 2 | Naive Matching
    p = 'needle'
    t = 'needle need noodle needle'
    occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
    assert occurrences == [0, 19], f"Naive Matching Example 2 failed: {occurrences}"
    assert num_alignments == 20, f"Naive Matching Example 2 failed: {num_alignments}"
    assert num_character_comparisons == 35, f"Naive Matching Example 2 failed: {num_character_comparisons}"
    print("Naive Matching Example 2 passed.")
    print(f"Occurrences: {occurrences}")
    print(f"Number of alignments tried: {num_alignments}")
    print(f"Number of character comparisons: {num_character_comparisons}\n")

if __name__ == "__main__":
    test_boyer_moore_and_naive_with_counts()

Boyer-Moore Example 1 passed.
Occurrences: [40]
Number of alignments tried: 12
Number of character comparisons: 15

Boyer-Moore Example 2 passed.
Occurrences: [0, 19]
Number of alignments tried: 5
Number of character comparisons: 18

Naive Matching Example 1 passed.
Occurrences: [40]
Number of alignments tried: 41
Number of character comparisons: 46

Naive Matching Example 2 passed.
Occurrences: [0, 19]
Number of alignments tried: 20
Number of character comparisons: 35



Question 1:
How many alignments does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

Question 2:
How many character comparisons does the naive exact matching algorithm try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

Question 3:
How many alignments does Boyer-Moore try when matching the string GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [13]:
t = readFasta('chr1.GRCh38.excerpt.fasta')

p = ('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG')
print("Pattern length:", len(p))

# Question 1
_, num_alignments_naive, _ = naive_with_counts(p, t)
print("Question 1:")
print(f"Number of alignments tried by naive exact matching: {num_alignments_naive}\n")

# Question 2
_, _, num_character_comparisons_naive = naive_with_counts(p, t)
print("Question 2:")
print(f"Number of character comparisons by naive exact matching: {num_character_comparisons_naive}\n")

# Question 3
alphabet = 'ACGT'
p_bm = BoyerMoore(p, alphabet)
_, num_alignments_bm, _ = boyer_moore_with_counts(p, p_bm, t)
print("Question 3:")
print(f"Number of alignments tried by Boyer-Moore: {num_alignments_bm}\n")


Pattern length: 47
Question 1:
Number of alignments tried by naive exact matching: 799954

Question 2:
Number of character comparisons by naive exact matching: 984143

Question 3:
Number of alignments tried by Boyer-Moore: 127974




Question 4:

Index-assisted approximate matching. In practicals, we built a Python class called IndexIndex

implementing an ordered-list version of the k-mer index.  The IndexIndex class is copied below.


In [14]:
# Given function

import bisect

class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using IndexIndex to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Download the Python module for building a k-mer index. 

https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

Write a function that, given a length-24 pattern P and given an IndexIndex object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAATGGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mmnaive_2mm function implemented in the previous module.

In [15]:
def approximate_match(P, T, max_mismatches, index):
    """Find approximate matches of P in T with up to max_mismatches using Index."""
    
    segment_length = len(P) // (max_mismatches + 1)
    all_matches = set()
    index_hits = 0
    
    for i in range(max_mismatches + 1):
        start = i * segment_length
        
        if i == max_mismatches:
            end = len(P)
        else:
            end = (i + 1) * segment_length
            
        p_segment = P[start:end]
  
        p_kmer = p_segment[:index.k]
        hits = index.query(p_kmer)
        index_hits += len(hits)
      
        for hit in hits:
            s = hit - start  
            if s < 0 or s + len(P) > len(T):
                continue 
            mismatches = 0
            for j in range(len(P)):
                if T[s + j] != P[j]:
                    mismatches += 1
                    if mismatches > max_mismatches:
                        break  
            if mismatches <= max_mismatches:
                all_matches.add(s)
    return list(all_matches), index_hits

In [16]:
# Question 4
p_approx = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 8
index = Index(t, k)
occurrences_approx, _ = approximate_match(p_approx, t, 2, index)
print("Question 4:")
print(f"Number of occurrences with up to 2 mismatches: {len(occurrences_approx)}\n")

Question 4:
Number of occurrences with up to 2 mismatches: 19



Question 5: Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of GGCGCGGTGGCTCACGCCTGTAATGGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?

  (Don't consider reverse complements.)

Hint: You should be able to use the boyer_mooreboyer_moore function (or the slower naivenaive function) to double-check your answer.

In [17]:
# Question 5
_, total_index_hits = approximate_match(p_approx, t, 2, index)
print("Question 5:")
print(f"Total number of index hits using Index: {total_index_hits}\n")

Question 5:
Total number of index hits using Index: 90



Question 6

Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA (first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222, and splitting into two subsequences of every other character could be represented as 121212

The following class SubseqIndexSubseqIndex is a more general implementation of IndexIndex that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [18]:
# Given function
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits


Write a function that, given a length-24 pattern P and given a SubseqIndexSubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

In [19]:
def query_subseq(p, t, subseq_ind):
    """Find all approximate occurrences of p in t with up to 2 mismatches,
    using the provided subsequence index."""
    
    occurrences = set()
    num_index_hits = 0
    
    for offset in range(3):
        subseq = p[offset : offset + subseq_ind.span : subseq_ind.ival]
        i = bisect.bisect_left(subseq_ind.index, (subseq, -1))
        
        hits = []
        
        while i < len(subseq_ind.index):
            if subseq_ind.index[i][0] != subseq:
                break
                
            hits.append(subseq_ind.index[i][1])
            i += 1
            
        num_index_hits += len(hits)
        
        for hit in hits:
            pos = hit - offset
            
            if pos < 0 or pos + len(p) > len(t):
                continue 
            mismatches = 0
            
            for i in range(len(p)):
                if t[pos + i] != p[i]:
                    mismatches += 1
                    
                    if mismatches > 2:
                        break  
                        
            if mismatches <= 2:
                occurrences.add(pos)
                
    return sorted(occurrences), num_index_hits

In [20]:
# Question 6
k_subseq = 8
ival = 3
subseq_ind = SubseqIndex(t, k_subseq, ival)
occurrences, num_index_hits = query_subseq(p_approx, t, subseq_ind)
print("Question 6:")
print(f"Total number of index hits using SubseqIndex: {num_index_hits}\n")

Question 6:
Total number of index hits using SubseqIndex: 79



# Module 3

We saw how to adapt dynamic programming to find approximate occurrences of a pattern in a text. Recall that:

    Rows of the dynamic programming matrix are labeled with bases from P and columns with bases from T

    Elements in the first row are set to 0

    Elements in the first column are set to 0, 1, 2, ..., as for edit distance

    Other elements are set in the same way as elements of a standard edit distance matrix

    The minimal value in the bottom row is the edit distance of the closest match between P and T

First, download the provided excerpt of human chromosome 1

Second, parse it using the readGenome function we wrote before.

Third, adapt the editDistance function we saw in practical (copied below) to answer questions 1 and 2 below. Your function should take arguments p (pattern), t (text) and should return the edit distance of the match between P and T with the fewest edits.

In [21]:
# Given function
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]


In [22]:
def approximateMatch(p, t):
    """Finds the minimal edit distance between pattern 'p' and any substring of text 't'."""
    
    m = len(p)
    n = len(t)
    
    previous_row = [0] * (n + 1)
    
    for i in range(1, m + 1):
        current_row = [i] + [0] * n
        
        for j in range(1, n + 1):
            
            if p[i - 1] == t[j - 1]:
                cost = 0
            else:
                cost = 1
                
            current_row[j] = min(
                previous_row[j] + 1,        
                current_row[j - 1] + 1,    
                previous_row[j - 1] + cost 
            )
            
        previous_row = current_row

    min_edit_distance = min(previous_row)
    
    return min_edit_distance

Question 1:
What is the edit distance of the best match between pattern GCTGATCGATCGTACGGCTGATCGATCGTACG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

Question 2:
What is the edit distance of the best match between pattern GATTTACCAGATTGAGGATTTACCAGATTGAG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [23]:
# Question 1
genome = readGenome('chr1.GRCh38.excerpt.fasta')
pattern1 = 'GCTGATCGATCGTACG'
distance1 = approximateMatch(pattern1, genome)
print("Question 1 Answer:")
print(distance1)

# Question 2
pattern2 = 'GATTTACCAGATTGAG'
distance2 = approximateMatch(pattern2, genome)
print("\nQuestion 2 Answer:")
print(distance2)

Question 1 Answer:
3

Question 2 Answer:
2



Question 3

In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.

In [24]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least kk bases long. To make an overlap graph, we could call overlap(a, b, min_length=k)overlap(a, b, min_length=k) on every possible pair of reads from the dataset.  Unfortunately, that will be very slow!

Consider this: Say we are using k=6, and we have a read aa whose length-6 suffix is GTCCTA.  Say GTCCTA does not occur in any other read in the dataset.  In other words, the 6-mer GTCCTA occurs at the end of read aa and nowhere else.  It follows that aa's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read aa and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of aa.  This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea.  You don't have to do it this way, but this might help you.  Let every k-mer in the dataset have an associated Python setset object, which starts out empty.  We use a Python dictionary to associate each k-mer with its corresponding setset. (1) For every k-mer in a read, we add the read to the setset object corresponding to that k-mer.  If our read is GATTAGATTA and k=3, we would add GATTAGATTA to the setset objects for GATGAT, ATTATT and TTATTA.  We do this for every read so that, at the end, each setset contains all reads containing the corresponding k-mer.  (2) Now, for each read aa, we find all overlaps involving a suffix of aa.  To do this, we take aa's length-k suffix, find all reads containing that k-mer (obtained from the corresponding setset) and call overlap(a, b, min_length=k)overlap(a, b, min_length=k) for each.

The most important point is that we do not call overlap(a, b, min_length=k)overlap(a, b, min_length=k) if bb does not contain the length-k suffix of a.

Download and parse the read sequences from the provided Phi-X FASTQ file. We'll just use their base sequences, so you can ignore read names and base qualities.  Also, no two reads in the FASTQ have the same sequence of bases.  This makes things simpler.


Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match.  Ignore reverse complements.

    Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than that.  (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.

    Hint 2: Remember not to overlap a read with itself. If you do, your answers will be too high.

    Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called overlap(a, b, min_length=k)overlap(a, b, min_length=k) on every pair of reads.  We also have provided a couple examples you can check against

    .

Picture the overlap graph corresponding to the overlaps just calculated.  How many edges are in the graph?  In other words, how many distinct pairs of reads overlap?

Question 4:
Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

In [25]:
# Question 3 and 4
sequences, qualities = readFastq('ERR266411_1.for_asm.fastq')

k = 30
kmer_dict = {}

for i, seq in enumerate(sequences):
    for j in range(len(seq) - k + 1):
        kmer = seq[j:j+k]
        
        if kmer not in kmer_dict:
            kmer_dict[kmer] = set()
            
        kmer_dict[kmer].add(i)
        
overlaps = {}
nodes_with_outgoing = set()

for i, seq in enumerate(sequences):
    suffix = seq[-k:]

    if suffix in kmer_dict:
        for j in kmer_dict[suffix]:
            if i != j:
                olen = overlap(seq, sequences[j], min_length=k)
                if olen > 0:
                    overlaps[(i, j)] = olen
                    nodes_with_outgoing.add(i)
                    
num_overlaps = len(overlaps)
print("\nQuestion 3 Answer:")
print(num_overlaps)
print("\nQuestion 4 Answer:")
print(len(nodes_with_outgoing))


Question 3 Answer:
904746

Question 4 Answer:
7161


# Module 4

Question 1:
In a practical, we saw the scs function (copied below along with overlapoverlap) for finding the shortest common superstring of a set of strings.

In [26]:
# Given functions
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's suffx in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) < len(shortest_sup):
            shortest_sup = sup  # found shorter superstring
    return shortest_sup  # return shortest


It's possible for there to be multiple different shortest common superstrings for the same set of input strings. Consider the input strings ABC, BCA, CAB. One shortest common superstring is ABCAB but another is BCABC and another is CABCA.

What is the length of the shortest common superstring of the following strings?

CCT, CTT, TGC, TGG, GAT, ATT

In [27]:
# Question 1
strings = ['CCT', 'CTT', 'TGC', 'TGG', 'GAT', 'ATT']
scs_result = scs(strings)
# Output the result and its length
print("Shortest common superstring:", scs_result)
print("Length:", len(scs_result))

Shortest common superstring: CCTTGGATTGC
Length: 11



Question 2:
How many different shortest common superstrings are there for the input strings given in the previous question?

Hint: You can modify the scs function to keep track of this. 


In [28]:
def scs_all(ss):
    
    """Returns a set of all shortest common superstrings of given strings."""
    
    shortest_sup_len = None
    shortest_sups = set()
    
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  
        
        for i in range(len(ssperm) - 1):
            olen = overlap(ssperm[i], ssperm[i + 1], min_length=1)
            sup += ssperm[i + 1][olen:]
            
        if (shortest_sup_len is None) or (len(sup) < shortest_sup_len):
            shortest_sup_len = len(sup)
            shortest_sups = {sup}
            
        elif len(sup) == shortest_sup_len:
            shortest_sups.add(sup)
            
    return shortest_sups

In [29]:
#Question 2
shortest_superstrings = scs_all(strings)
print("\nQuestion 2 Answer:")
print("Length of the shortest common superstring:", len(next(iter(shortest_superstrings))))


Question 2 Answer:
Length of the shortest common superstring: 11


Question 3:
Download this FASTQ file containing synthetic sequencing reads from a mystery virus:

All the reads are the same length (100 bases) and are exact copies of substrings from the forward strand of the virus genome.  You don't have to worry about sequencing errors, ploidy, or reads coming from the reverse strand.

Assemble these reads using one of the approaches discussed, such as greedy shortest common superstring.  Since there are many reads, you might consider ways to make the algorithm faster, such as the one discussed in the programming assignment in the previous module.

How many As are there in the full, assembled genome?

Hint: the virus genome you are assembling is exactly 15,894 bases long

Question 4:
How many Ts are there in the full, assembled genome from the previous question?

In [30]:
def build_kmer_dict(reads, k):
    
    """Builds a dictionary mapping each k-mer to a set of reads containing that k-mer."""
    
    kmer_dict = {}
    
    for read in reads:
        for i in range(len(read) - k + 1):
            kmer = read[i:i+k]
            
            if kmer in kmer_dict:
                kmer_dict[kmer].add(read)
            else:
                kmer_dict[kmer] = set([read])
                
    return kmer_dict

def pick_maximal_overlap(reads, kmer_dict, k):
    
    """Finds the pair of reads with the maximal suffix/prefix overlap."""
    
    read_a, read_b = None, None
    best_olen = 0
    
    for a in reads:
        suffix = a[-k:]
        if suffix in kmer_dict:
            candidates = kmer_dict[suffix]
            
            for b in candidates:
                if a != b:
                    olen = overlap(a, b, min_length=k)
                    if olen > best_olen:
                        best_olen = olen
                        read_a, read_b = a, b
                        
    return read_a, read_b, best_olen

def greedy_scs(reads, k):
    
    """Assemble the genome using a greedy shortest common superstring approach with k-mer optimization."""
    
    reads = reads.copy() 
    
    while True:
        kmer_dict = build_kmer_dict(reads, k)
        read_a, read_b, olen = pick_maximal_overlap(reads, kmer_dict, k)
        
        if olen == 0:
            break
            
        reads.remove(read_a)
        reads.remove(read_b)
        merged_read = read_a + read_b[olen:]
        reads.append(merged_read)

    assembled_genome = ''.join(reads)
    
    return assembled_genome

def count_bases(sequence):
    
    """Count the number of each base in the sequence."""
    
    counts = {}
    
    for base in sequence:
        if base in counts:
            counts[base] += 1
        else:
            counts[base] = 1
            
    return counts

In [31]:
filename = 'ads1_week4_reads.fq'

sequences, qualities = readFastq(filename)
sequences, qualities = readFastq(filename)
reads = sequences
print("Total reads:", len(reads))

assembled_genome = greedy_scs(reads, k=30)
print("Length of assembled genome:", len(assembled_genome))

base_counts = count_bases(assembled_genome)
num_As = base_counts.get('A', 0)
num_Ts = base_counts.get('T', 0)
print("Number of As:", num_As)
print("Number of Ts:", num_Ts)

Total reads: 1881
Length of assembled genome: 15894
Number of As: 4633
Number of Ts: 3723
